In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import pickle

In [2]:
## load the ann trained model, scaler pkl & OHE encoder pkl
model= load_model("model.h5")

## load encoder and scaler
with open('OHE_encoder_geo.pkl','rb') as file:
    label_geo_encoder=pickle.load(file)

with open('label_gender_encoder.pkl','rb') as file:
    label_gender_encoder=pickle.load(file)

with open('Scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [3]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 30,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}


In [16]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,30,60000,2,1,1,50000


In [8]:
geo_encoded = label_geo_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_geo_encoder.get_feature_names_out(['Geography']))
geo_encoded_df


c:\Users\ashri\Desktop\Prashu\Gen AI\ANN\annenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [17]:
input_df['Gender'] = label_gender_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,30,60000,2,1,1,50000


In [18]:
# Combine one-hot encoded columns with input data
input_df = pd.concat([input_df.drop('Geography', axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,30,60000,2,1,1,50000,1.0,0.0,0.0


In [20]:
## Scaling the data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.51222865,  0.90636285,  0.10591292,  8.64688662, -0.26147196,
         0.8016426 ,  0.64900815,  0.97238125, -0.86754814,  0.99575899,
        -0.57812007, -0.57330877]])

In [22]:
## Predict
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 49ms/step


array([[0.00225874]], dtype=float32)

In [23]:
prediction_prob = prediction[0][0]
prediction_prob

0.0022587432

In [24]:

if prediction_prob > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
